In [1]:
import os
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

DRIVE_PATH = '/content/gdrive/My\ Drive/cs285_f2023'
DRIVE_PYTHON_PATH = DRIVE_PATH.replace('\\', '')
if not os.path.exists(DRIVE_PYTHON_PATH):
  %mkdir $DRIVE_PATH

## the space in `My Drive` causes some issues,
## make a symlink to avoid this
SYM_PATH = '/content/cs285_f2023'
if not os.path.exists(SYM_PATH):
  !ln -s $DRIVE_PATH $SYM_PATH

!apt update
!apt install -y --no-install-recommends \
        build-essential \
        curl \
        git \
        gnupg2 \
        make \
        cmake \
        ffmpeg \
        swig \
        libz-dev \
        unzip \
        zlib1g-dev \
        libglfw3 \
        libglfw3-dev \
        libxrandr2 \
        libxinerama-dev \
        libxi6 \
        libxcursor-dev \
        libgl1-mesa-dev \
        libgl1-mesa-glx \
        libglew-dev \
        libosmesa6-dev \
        lsb-release \
        ack-grep \
        patchelf \
        wget \
        xpra \
        xserver-xorg-dev \
        ffmpeg
!apt-get install python-opengl -y
!apt install xvfb -y

%cd $SYM_PATH
!git clone https://github.com/geekLucian/cs285-deeprl.git homework_fall2023 
%cd homework_fall2023/hw2
%pip install -r requirements.txt
%pip install -e .

Mounted at /content/gdrive
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,260 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,283 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://arc

In [1]:
import time

from cs285.scripts.run_hw2 import run_training_loop

%load_ext autoreload
%autoreload 2

/usr/local/lib/python3.10/dist-packages/google/rpc/__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  pkg_resources.declare_namespace(__name__)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2349: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(parent)


In [27]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--env_name", type=str, required=True)
parser.add_argument("--exp_name", type=str, required=True)
parser.add_argument("--n_iter", "-n", type=int, default=200)

parser.add_argument("--use_reward_to_go", "-rtg", action="store_true")
parser.add_argument("--use_baseline", action="store_true")
parser.add_argument("--baseline_learning_rate", "-blr", type=float, default=5e-3)
parser.add_argument("--baseline_gradient_steps", "-bgs", type=int, default=5)
parser.add_argument("--gae_lambda", type=float, default=None)
parser.add_argument("--normalize_advantages", "-na", action="store_true")
parser.add_argument(
    "--batch_size", "-b", type=int, default=1000
)  # steps collected per train iteration
parser.add_argument(
    "--eval_batch_size", "-eb", type=int, default=400
)  # steps collected per eval iteration

parser.add_argument("--discount", type=float, default=1.0)
parser.add_argument("--learning_rate", "-lr", type=float, default=5e-3)
parser.add_argument("--n_layers", "-l", type=int, default=2)
parser.add_argument("--layer_size", "-s", type=int, default=64)

parser.add_argument(
    "--ep_len", type=int
)  # students shouldn't change this away from env's default
parser.add_argument("--seed", type=int, default=1)
parser.add_argument("--no_gpu", "-ngpu", action="store_true")
parser.add_argument("--which_gpu", "-gpu_id", default=0)
parser.add_argument("--video_log_freq", type=int, default=-1)
parser.add_argument("--scalar_log_freq", type=int, default=1)

parser.add_argument("--action_noise_std", type=float, default=0)

# Experiment 1
args_string_1_1 = "--env_name CartPole-v0 -n 100 -b 1000 --exp_name cartpole"
args_string_1_2 = "--env_name CartPole-v0 -n 100 -b 1000 -rtg --exp_name cartpole_rtg"
args_string_1_3 = "--env_name CartPole-v0 -n 100 -b 1000 -na --exp_name cartpole_na"
args_string_1_4 = "--env_name CartPole-v0 -n 100 -b 1000 -rtg -na --exp_name cartpole_rtg_na"
args_string_1_5 = "--env_name CartPole-v0 -n 100 -b 4000 --exp_name cartpole_lb"
args_string_1_6 = "--env_name CartPole-v0 -n 100 -b 4000 -rtg --exp_name cartpole_lb_rtg"
args_string_1_7 = "--env_name CartPole-v0 -n 100 -b 4000 -na --exp_name cartpole_lb_na"
args_string_1_8 = "--env_name CartPole-v0 -n 100 -b 4000 -rtg -na --exp_name cartpole_lb_rtg_na"
args_strings_1 = [
    args_string_1_1,
    args_string_1_2,
    args_string_1_3,
    args_string_1_4,
    args_string_1_5,
    args_string_1_6,
    args_string_1_7,
    args_string_1_8
]

# Experiment 2
args_string_2_1 =  "--env_name HalfCheetah-v4 -n 100 -b 5000 -rtg --discount 0.95 -lr 0.01 --exp_name cheetah"
args_string_2_2 = "--env_name HalfCheetah-v4 -n 100 -b 5000 -rtg --discount 0.95 -lr 0.01 --use_baseline -blr 0.01 -bgs 5 --exp_name cheetah_baseline"
args_strings_2 = [args_string_2_1, args_string_2_2]

# Experiment 3
args_string_3_1 = "--env_name LunarLander-v2 --ep_len 1000 --discount 0.99 -n 300 -l 3 -s 128 -b 2000 -lr 0.001 --use_reward_to_go --use_baseline --gae_lambda 0 --exp_name lunar_lander_lambda000"
args_string_3_2 = "--env_name LunarLander-v2 --ep_len 1000 --discount 0.99 -n 300 -l 3 -s 128 -b 2000 -lr 0.001 --use_reward_to_go --use_baseline --gae_lambda 0.95 --exp_name lunar_lander_lambda095"
args_string_3_3 = "--env_name LunarLander-v2 --ep_len 1000 --discount 0.99 -n 300 -l 3 -s 128 -b 2000 -lr 0.001 --use_reward_to_go --use_baseline --gae_lambda 0.98 --exp_name lunar_lander_lambda098"
args_string_3_4 = "--env_name LunarLander-v2 --ep_len 1000 --discount 0.99 -n 300 -l 3 -s 128 -b 2000 -lr 0.001 --use_reward_to_go --use_baseline --gae_lambda 0.99 --exp_name lunar_lander_lambda099"
args_string_3_5 = "--env_name LunarLander-v2 --ep_len 1000 --discount 0.99 -n 300 -l 3 -s 128 -b 2000 -lr 0.001 --use_reward_to_go --use_baseline --gae_lambda 1 --exp_name lunar_lander_lambda100"
args_strings_3 = [args_string_3_1, args_string_3_2, args_string_3_3, args_string_3_4, args_string_3_5]

In [34]:
import os
import pickle

# create directory for logging
logdir_prefix = "q2_pg_"  # keep for autograder
logs_dict = {}
data_path ='/content/cs285_f2023/hw2/data'

if not (os.path.exists(data_path)):
    os.makedirs(data_path)


/content/cs285_f2023/hw2/data/q2_pg_lunar_lander_lambda0.99_LunarLander-v2_25-09-2023_06-38-00
########################
logging outputs to  /content/cs285_f2023/hw2/data/q2_pg_lunar_lander_lambda0.99_LunarLander-v2_25-09-2023_06-38-00
########################
Using GPU id 0

********** Iteration 0 ************
[INFO] sample No.0 trajectory

Eval_AverageReturn : -6.762535095214844
Eval_StdReturn : 7.325084209442139
Eval_MaxReturn : 9.932437896728516
Eval_MinReturn : -18.90288734436035
Eval_AverageEpLen : 10.0
Train_AverageReturn : -4.88976526260376
Train_StdReturn : 7.153604984283447
Train_MaxReturn : 10.241032600402832
Train_MinReturn : -15.969182014465332
Train_AverageEpLen : 10.0
Actor Loss : -805.3447265625
Baseline Loss : 31.41647720336914
Train_EnvstepsSoFar : 200
TimeSinceStart : 0.4610862731933594
Initial_DataCollection_AverageReturn : -4.88976526260376
Done logging...



********** Iteration 1 ************
[INFO] sample No.0 trajectory

Eval_AverageReturn : -6.625512599945068
E

In [ ]:
#@title Experiment 1
for args_string in args_strings_1:
  args = parser.parse_args(args_string.split())
  logdir = (
      logdir_prefix
      + args.exp_name
      + "_"
      + args.env_name
      + "_"
      + time.strftime("%d-%m-%Y_%H-%M-%S")
  )
  logdir = os.path.join(data_path, logdir)
  args.logdir = logdir
  if not (os.path.exists(logdir)):
      os.makedirs(logdir)
  # traning
  print(args.logdir)
  full_logs = run_training_loop(args)
  logs_dict[args.exp_name] = full_logs
  with open(os.path.join(data_path, 'e1_logs_dict.pkl'), 'wb') as f:
        pickle.dump(logs_dict, f)

In [ ]:
#@title Experiment 2
for args_string in args_strings_2:
  args = parser.parse_args(args_string.split())
  logdir = (
      logdir_prefix
      + args.exp_name
      + "_"
      + args.env_name
      + "_"
      + time.strftime("%d-%m-%Y_%H-%M-%S")
  )
  logdir = os.path.join(data_path, logdir)
  args.logdir = logdir
  if not (os.path.exists(logdir)):
      os.makedirs(logdir)
  # traning
  print(args.logdir)
  full_logs = run_training_loop(args)
  logs_dict[args.exp_name] = full_logs
  with open(os.path.join(data_path, 'e2_logs_dict.pkl'), 'wb') as f:
        pickle.dump(logs_dict, f)

In [ ]:
#@title Experiment 3
for args_string in args_strings_3:
  args = parser.parse_args(args_string.split())
  logdir = (
      logdir_prefix
      + args.exp_name
      + "_"
      + args.env_name
      + "_"
      + time.strftime("%d-%m-%Y_%H-%M-%S")
  )
  logdir = os.path.join(data_path, logdir)
  args.logdir = logdir
  if not (os.path.exists(logdir)):
      os.makedirs(logdir)
  # traning
  print(args.logdir)
  full_logs = run_training_loop(args)
  logs_dict[args.exp_name] = full_logs
  with open(os.path.join(data_path, 'e3_logs_dict.pkl'), 'wb') as f:
        pickle.dump(logs_dict, f)

In [25]:
with open(os.path.join(data_path, 'logs_dict.pkl'), 'rb') as f:
  loaded_data = pickle.load(f)


OrderedDict([('Eval_AverageReturn', 25.75), ('Eval_StdReturn', 16.909687), ('Eval_MaxReturn', 88.0), ('Eval_MinReturn', 14.0), ('Eval_AverageEpLen', 25.75), ('Train_AverageReturn', 26.15), ('Train_StdReturn', 13.823078), ('Train_MaxReturn', 59.0), ('Train_MinReturn', 9.0), ('Train_AverageEpLen', 26.15), ('Actor Loss', array(-4.9936104, dtype=float32)), ('Train_EnvstepsSoFar', 1046), ('TimeSinceStart', 0.8918595314025879), ('Initial_DataCollection_AverageReturn', 26.15)])


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
!pip install -Uqq ipdb
import ipdb

In [5]:
# import importlib
# importlib.reload(cs285)
%pdb off
%load_ext autoreload
%autoreload 2

Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
## run training
print(args.logdir)
run_training_loop(args)

/content/cs285_f2023/hw2/data/q2_pg_cartpole_CartPole-v0_25-09-2023_03-56-53
########################
logging outputs to  /content/cs285_f2023/hw2/data/q2_pg_cartpole_CartPole-v0_25-09-2023_03-56-53
########################
Using GPU id 0

********** Iteration 0 ************
[INFO] sample No.0 trajectory
[INFO] sample No.200 trajectory
[INFO] sample No.400 trajectory
[INFO] sample No.600 trajectory
[INFO] sample No.800 trajectory

Eval_AverageReturn : 33.66666793823242
Eval_StdReturn : 20.038850784301758
Eval_MaxReturn : 79.0
Eval_MinReturn : 10.0
Eval_AverageEpLen : 33.666666666666664
Train_AverageReturn : 23.52899932861328
Train_StdReturn : 12.341360092163086
Train_MaxReturn : 127.0
Train_MinReturn : 9.0
Train_AverageEpLen : 23.529
Actor Loss : 486447.9375
Train_EnvstepsSoFar : 23529
TimeSinceStart : 14.97639012336731
Initial_DataCollection_AverageReturn : 23.52899932861328
Done logging...



********** Iteration 1 ************
[INFO] sample No.0 trajectory
[INFO] sample No.200 traje

KeyboardInterrupt: ignored

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/cs285_f2023/hw2/data